In [ ]:
from google.colab import drive
drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
np.random.seed (1111)
import matplotlib.pyplot as plt
%matplotlib inline
import keras
from  keras.models import Model
from  keras.layers import *
from  keras import optimizers
from  sklearn.model_selection import train_test_split
from  keras.utils.np_utils import to_categorical

In [ ]:
df_train = pd.read_csv ('/content/drive/MyDrive/Content/Titanick/train.csv') 
df_test = pd.read_csv ('/content/drive/MyDrive/Content/Titanick/test.csv') 
df_ty = pd.read_csv ('/content/drive/MyDrive/Content/Titanick/gender_submission.csv') 
df_train = df_train.reindex(columns=['Survived', 'PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'])

In [ ]:
df_train.describe()

,Survived,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,0.383838,446.000000,2.308642,29.699118,0.523008,0.381594,32.204208
std,0.486592,257.353842,0.836071,14.526497,1.102743,0.806057,49.693429
min,0.000000,1.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,0.000000,223.500000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,0.000000,446.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,1.000000,668.500000,3.000000,38.000000,1.000000,0.000000,31.000000
max,1.000000,891.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [ ]:
df_test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [ ]:
df_train.isna().sum()

Survived         0
PassengerId      0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [ ]:
df_test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [ ]:
df_train['Embarked'].fillna(df_train['Embarked'].value_counts().idxmax(), inplace=True)

In [ ]:
df_train['Age'].fillna(df_train['Age'].value_counts().idxmax(), inplace=True)
df_test['Age'].fillna(df_test['Age'].value_counts().idxmax(), inplace=True)
df_test['Fare'].fillna(df_test['Fare'].mean(), inplace=True)

In [ ]:
df_train

,Survived,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,1,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,0,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,0,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,1,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,0,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,24.0,1,2,W./C. 6607,23.4500,NaN,S
889,1,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
df_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,21.0,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,21.0,0,0,359309,8.0500,NaN,S


In [ ]:
df_ty

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [ ]:
X_train = df_train.iloc[:, 1:12]
y_train = df_train.iloc[:, 0]

In [ ]:
X_train

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,24.0,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [ ]:
y_train

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64

In [ ]:
X_test = df_test.iloc[:, 0:11]
y_test = df_ty.iloc[:, 1]

In [ ]:
#X_train, X_cv, y_train, y_cv = train_test_split(df_features, df_label,
#                                                test_size = 0.2,
#                                                random_state = 1111)
#X_train = X_train.values.reshape(713, 11)
#x_cv = X_cv.values.reshape(178, 11) 

In [ ]:
X_test

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,21.0,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,21.0,0,0,359309,8.0500,NaN,S


In [ ]:
y_test

0      0
1      1
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    0
Name: Survived, Length: 418, dtype: int64

In [ ]:
g=0
for i in range(len(y_train)):
  if y_train[i]==1:
    g=g+1
print(g)

342


In [ ]:
S = X_train.Ticket.to_string().replace('[','').replace(']','')
for i in range(len(X_train.PassengerId)):
  X_train.Ticket[i] = "".join(c for c in X_train.Ticket[i] if  c.isdecimal())
  if X_train.Age[i] == 'NaN':
    X_train.Age[i] = "NaN"
  elif X_train.Age[i] < 20:
    X_train.Age[i] = "Yng"
  elif (X_train.Age[i] >= 20) and (X_train.Age[i] <= 40):
    X_train.Age[i] = "Mid"
  elif (X_train.Age[i] > 40):
    X_train.Age[i] = "Old"
#for i in range(len(X_train.PassengerId)):   
#  X_train.Ticket = int(X_train.Ticket)
X_train = pd.get_dummies(X_train, columns=['Embarked', 'Sex', 'Age'])
#del X_train['Cabin']
X_train.Name = X_train.Name.str.len() 
print(X_train)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

     PassengerId  Pclass  Name  SibSp  Parch    Ticket     Fare Cabin  \
0              1       3    23      1      0    521171   7.2500   NaN   
1              2       1    51      1      0     17599  71.2833   C85   
2              3       3    22      0      0  23101282   7.9250   NaN   
3              4       1    44      1      0    113803  53.1000  C123   
4              5       3    24      0      0    373450   8.0500   NaN   
..           ...     ...   ...    ...    ...       ...      ...   ...   
886          887       2    21      0      0    211536  13.0000   NaN   
887          888       1    28      0      0    112053  30.0000   B42   
888          889       3    40      1      2      6607  23.4500   NaN   
889          890       1    21      0      0    111369  30.0000  C148   
890          891       3    19      0      0    370376   7.7500   NaN   

     Embarked_C  Embarked_Q  Embarked_S  Sex_female  Sex_male  Age_Mid  \
0             0           0           1          

In [ ]:
S = X_test.Ticket.to_string().replace('[','').replace(']','')
for i in range(len(X_test.PassengerId)):
  X_test.Ticket[i] = int("".join(c for c in X_test.Ticket[i] if  c.isdecimal()))
  if X_test.Age[i] == 'NaN':
    X_test.Age[i] = "NaN"
  elif X_test.Age[i] < 20:
    X_test.Age[i] = "Yng"
  elif (X_test.Age[i] >= 20) and (X_test.Age[i] <= 40):
    X_test.Age[i] = "Mid"
  elif (X_test.Age[i] > 40):
    X_test.Age[i] = "Old"
X_test = pd.get_dummies(X_test, columns=['Embarked', 'Sex', 'Age'])
#del X_test['Cabin']
X_test = X_test.reindex(X_train.columns , axis = 'columns', fill_value = 0)
X_test.Name = X_test.Name.str.len() 
print(X_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

     PassengerId  Pclass  Name  SibSp  Parch   Ticket      Fare Cabin  \
0            892       3    16      0      0   330911    7.8292   NaN   
1            893       3    32      1      0   363272    7.0000   NaN   
2            894       2    25      0      0   240276    9.6875   NaN   
3            895       3    16      0      0   315154    8.6625   NaN   
4            896       3    44      1      1  3101298   12.2875   NaN   
..           ...     ...   ...    ...    ...      ...       ...   ...   
413         1305       3    18      0      0    53236    8.0500   NaN   
414         1306       1    28      0      0    17758  108.9000  C105   
415         1307       3    28      0      0  3101262    7.2500   NaN   
416         1308       3    19      0      0   359309    8.0500   NaN   
417         1309       3    24      1      1     2668   22.3583   NaN   

     Embarked_C  Embarked_Q  Embarked_S  Sex_female  Sex_male  Age_Mid  \
0             0           1           0          

In [ ]:
X_train

,PassengerId,Pclass,Name,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Age_Mid,Age_Old,Age_Yng
0,1,3,23,1,0,521171,7.2500,NaN,0,0,1,0,1,1,0,0
1,2,1,51,1,0,17599,71.2833,C85,1,0,0,1,0,1,0,0
2,3,3,22,0,0,23101282,7.9250,NaN,0,0,1,1,0,1,0,0
3,4,1,44,1,0,113803,53.1000,C123,0,0,1,1,0,1,0,0
4,5,3,24,0,0,373450,8.0500,NaN,0,0,1,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,2,21,0,0,211536,13.0000,NaN,0,0,1,0,1,1,0,0
887,888,1,28,0,0,112053,30.0000,B42,0,0,1,1,0,0,0,1
888,889,3,40,1,2,6607,23.4500,NaN,0,0,1,1,0,1,0,0
889,890,1,21,0,0,111369,30.0000,C148,1,0,0,0,1,1,0,0


In [ ]:
X_test

,PassengerId,Pclass,Name,SibSp,Parch,Ticket,Fare,Cabin,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Age_Mid,Age_Old,Age_Yng
0,892,3,16,0,0,330911,7.8292,NaN,0,1,0,0,1,1,0,0
1,893,3,32,1,0,363272,7.0000,NaN,0,0,1,1,0,0,1,0
2,894,2,25,0,0,240276,9.6875,NaN,0,1,0,0,1,0,1,0
3,895,3,16,0,0,315154,8.6625,NaN,0,0,1,0,1,1,0,0
4,896,3,44,1,1,3101298,12.2875,NaN,0,0,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,18,0,0,53236,8.0500,NaN,0,0,1,0,1,1,0,0
414,1306,1,28,0,0,17758,108.9000,C105,1,0,0,1,0,1,0,0
415,1307,3,28,0,0,3101262,7.2500,NaN,0,0,1,0,1,1,0,0
416,1308,3,19,0,0,359309,8.0500,NaN,0,0,1,0,1,1,0,0


In [ ]:
print("Максимум и минимум: ", max(X_train.Fare), " ", min(X_train.Fare))
print("Сред. Арифм: ", sum(X_train.Fare)/891)

Максимум и минимум:  512.3292   0.0
Сред. Арифм:  32.2042079685746


In [ ]:
X_train.dtypes

PassengerId      int64
Pclass           int64
Name             int64
SibSp            int64
Parch            int64
Ticket          object
Fare           float64
Cabin           object
Embarked_C       uint8
Embarked_Q       uint8
Embarked_S       uint8
Sex_female       uint8
Sex_male         uint8
Age_Mid          uint8
Age_Old          uint8
Age_Yng          uint8
dtype: object

In [ ]:
del X_train['Cabin']
del X_test['Cabin']
del X_train['Ticket']
del X_test['Ticket']

In [ ]:
#X_train=X_train.dropna()
#X_train=X_train.dropna(axis=0)
#X_train=X_train.dropna().reset_index(drop=True)
#X_test=X_test.dropna()
#X_test=X_test.dropna(axis=0)
#X_train=X_train.dropna().reset_index(drop=True)

In [ ]:
X_train = X_train.astype('float32'); y_train.astype('float32'); X_test.astype('float32'); y_test.astype('float32')

num_digits = 2
y_train = to_categorical(y_train, num_digits)
y_test = to_categorical(y_test, num_digits)

In [ ]:
X_test

,PassengerId,Pclass,Name,SibSp,Parch,Fare,Embarked_C,Embarked_Q,Embarked_S,Sex_female,Sex_male,Age_Mid,Age_Old,Age_Yng
0,892,3,16,0,0,7.8292,0,1,0,0,1,1,0,0
1,893,3,32,1,0,7.0000,0,0,1,1,0,0,1,0
2,894,2,25,0,0,9.6875,0,1,0,0,1,0,1,0
3,895,3,16,0,0,8.6625,0,0,1,0,1,1,0,0
4,896,3,44,1,1,12.2875,0,0,1,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,18,0,0,8.0500,0,0,1,0,1,1,0,0
414,1306,1,28,0,0,108.9000,1,0,0,1,0,1,0,0
415,1307,3,28,0,0,7.2500,0,0,1,0,1,1,0,0
416,1308,3,19,0,0,8.0500,0,0,1,0,1,1,0,0


In [ ]:
n_input = 14
n_hidden_1 = 56  
n_hidden_2 = 28
num_digits = 2

In [ ]:
Inp = Input(shape = (14,))
x = Dense(n_hidden_1, activation='swish', name = 'hidden_layer_1')(Inp)
x = Dense(n_hidden_2, activation='swish', name = 'hidden_layer_2')(x)
output = Dense(num_digits, activation='softmax', name = 'out_layer')(x)

In [ ]:
model = Model(Inp, output)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 14)]              0         
                                                                 
 hidden_layer_1 (Dense)      (None, 56)                840       
                                                                 
 hidden_layer_2 (Dense)      (None, 28)                1596      
                                                                 
 out_layer (Dense)           (None, 2)                 58        
                                                                 
Total params: 2,494
Trainable params: 2,494
Non-trainable params: 0
_________________________________________________________________


In [ ]:
learning_rate = 0.1
training_epochs = 40
batch_size = 25
rmsprop = tf.keras.optimizers.RMSprop(learning_rate)

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer = 'rmsprop',
              metrics=['accuracy', tf.keras.metrics.RootMeanSquaredError()]) 

In [ ]:
history1 = model.fit(X_train, y_train,
                      batch_size = batch_size,
                      epochs = training_epochs,
                      verbose = 2,
                      validation_data=(X_test, y_test))

Epoch 1/40
36/36 - 1s - loss: 10.7833 - accuracy: 0.4961 - root_mean_squared_error: 0.6790 - val_loss: 3.9740 - val_accuracy: 0.3636 - val_root_mean_squared_error: 0.7881 - 1s/epoch - 33ms/step
Epoch 2/40
36/36 - 0s - loss: 3.1785 - accuracy: 0.5522 - root_mean_squared_error: 0.6244 - val_loss: 5.5907 - val_accuracy: 0.6507 - val_root_mean_squared_error: 0.5892 - 93ms/epoch - 3ms/step
Epoch 3/40
36/36 - 0s - loss: 3.2203 - accuracy: 0.5735 - root_mean_squared_error: 0.6137 - val_loss: 16.0687 - val_accuracy: 0.6364 - val_root_mean_squared_error: 0.6030 - 99ms/epoch - 3ms/step
Epoch 4/40
36/36 - 0s - loss: 2.4987 - accuracy: 0.6117 - root_mean_squared_error: 0.5789 - val_loss: 2.7514 - val_accuracy: 0.6483 - val_root_mean_squared_error: 0.5868 - 102ms/epoch - 3ms/step
Epoch 5/40
36/36 - 0s - loss: 2.9670 - accuracy: 0.5836 - root_mean_squared_error: 0.6010 - val_loss: 16.2315 - val_accuracy: 0.3636 - val_root_mean_squared_error: 0.7977 - 132ms/epoch - 4ms/step
Epoch 6/40
36/36 - 0s - lo

In [ ]:
test_pred = pd.DataFrame(model.predict(X_test, batch_size=30))
test_pred = pd.DataFrame(test_pred.idxmax(axis = 1))
test_pred.index.name = 'Index'
test_pred = test_pred.rename(columns = {0: 'Model pred'})
test_pred["Labels"] = df_ty.Survived
print(test_pred)

       Model pred  Labels
Index                    
0               0       0
1               0       1
2               0       0
3               0       0
4               0       1
...           ...     ...
413             0       0
414             1       1
415             0       0
416             0       0
417             0       0

[418 rows x 2 columns]


In [ ]:
test_csv = pd.DataFrame(model.predict(X_test, batch_size=30))
test_csv = pd.DataFrame(test_csv.idxmax(axis = 1))
test_csv.index.name = 'PassengerId'
test_csv = test_csv.rename(columns = {0: 'Survived'})
print(test_csv)
test_csv.to_csv('/content/drive/MyDrive/Content/Titanick/prediction.csv')

             Survived
PassengerId          
0                   0
1                   0
2                   0
3                   0
4                   0
...               ...
413                 0
414                 1
415                 0
416                 0
417                 0

[418 rows x 1 columns]


In [ ]:
####################################################
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

forest_model = RandomForestRegressor(random_state=1)
forest_model.fit(X_train, y_train)
melb_preds = forest_model.predict(X_test)
print(mean_absolute_error(y_test, melb_preds))

0.2733492822966506
